Promedio Parametros

In [ ]:
import pandas as pd
import zipfile
import os

# Reasignar la ruta del archivo después del reinicio
excel_pathAntioquia1 = r"C:\Users\Stefania Alvarez\Documents\Universidad\2025-1\Tesis Final\Modelos de prediccion\Antioquia1-Final.xlsx"
excel_pathAntioquia2 = r"C:\Users\Stefania Alvarez\Documents\Universidad\2025-1\Tesis Final\Modelos de prediccion\Antioquia2-Final.xlsx"

excel_pathAtlantico1 = r"C:\Users\Stefania Alvarez\Documents\Universidad\2025-1\Tesis Final\Modelos de prediccion\Atlantico1-Final.xlsx"
excel_pathAtlantico2 = r"C:\Users\Stefania Alvarez\Documents\Universidad\2025-1\Tesis Final\Modelos de prediccion\Atlantico1-Final.xlsx"

excel_pathCordoba1 = r"C:\Users\Stefania Alvarez\Documents\Universidad\2025-1\Tesis Final\Modelos de prediccion\Cordoba1-Final.xlsx"

excel_pathGuajira1 = r"C:\Users\Stefania Alvarez\Documents\Universidad\2025-1\Tesis Final\Modelos de prediccion\Guajira1-Final.xlsx"
excel_pathGuajira2 = r"C:\Users\Stefania Alvarez\Documents\Universidad\2025-1\Tesis Final\Modelos de prediccion\Guajira2-Final.xlsx"
excel_pathGuajira3 = r"C:\Users\Stefania Alvarez\Documents\Universidad\2025-1\Tesis Final\Modelos de prediccion\Guajira3-Final.xlsx"


new_excel_path = r"C:\Users\Stefania Alvarez\Documents\Universidad\2025-1\Tesis Final\Modelos de prediccion\PruebaDatos.xlsx"

In [ ]:
exceles = [excel_pathAntioquia1, excel_pathAntioquia2, excel_pathAtlantico1, excel_pathAtlantico2, excel_pathCordoba1, excel_pathGuajira1, excel_pathGuajira2, excel_pathGuajira3 ]
lugares = {"Antioquia1":excel_pathAntioquia1, "Antioquia2":excel_pathAntioquia2, "Atlantico1":excel_pathAtlantico1, "Atlantico2":excel_pathAtlantico2, "Cordoba1":excel_pathCordoba1, "Guajira1":excel_pathGuajira1,"Guajira2": excel_pathGuajira2,"Guajira3": excel_pathGuajira3}


In [ ]:
import pandas as pd
import os

for nombre, excel_path in lugares.items():
    all_sheets = pd.read_excel(excel_path, sheet_name=None)

    averages_df = pd.DataFrame()

    for sheet_name, df in all_sheets.items():
        numeric_df = df.select_dtypes(include='number')
        means = numeric_df.mean().round(2)
        means['Hoja'] = sheet_name
        averages_df = pd.concat([averages_df, means.to_frame().T], ignore_index=True)

    # Ordenar columnas
    cols = ['Hoja'] + [col for col in averages_df.columns if col != 'Hoja']
    averages_df = averages_df[cols]

    # Limpiar nombre para la hoja de Excel
    safe_sheet_name = f"Resumen_{nombre}"[:31]  # Excel solo permite 31 caracteres

    with pd.ExcelWriter(new_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        averages_df.to_excel(writer, sheet_name=safe_sheet_name, index=False)


Extraer WSO

In [ ]:
import os
import pandas as pd

# Directorio donde están los archivos Excel
input_dir = r"C:\Users\Stefania Alvarez\Documents\Universidad\2025-1\Tesis Final\Modelos de prediccion"
output_path = os.path.join(input_dir, "PruebaDatos.xlsx")

# Listar todos los archivos que comienzan con 'Resultados_LINTUL2_' y terminan en '.xlsx'
excel_files = [f for f in os.listdir(input_dir) if f.startswith("Resultados_LINTUL2_") and f.endswith(".xlsx")]

# Inicializar un escritor de Excel
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    for file in excel_files:
        file_path = os.path.join(input_dir, file)
        
        # Leer todas las hojas del archivo actual
        all_sheets = pd.read_excel(file_path, sheet_name=None)
        
        # Inicializar un DataFrame base con la columna 'time'
        merged_df = pd.DataFrame()

        for sheet_name, df in all_sheets.items():
            if 'time' in df.columns and 'WSO' in df.columns:
                temp_df = df[['time', 'WSO']].copy()
                temp_df = temp_df.rename(columns={'WSO': sheet_name})  # renombrar 'WSO' como nombre de la hoja

                if merged_df.empty:
                    merged_df = temp_df
                else:
                    merged_df = pd.merge(merged_df, temp_df, on='time', how='outer')  # juntar por 'time'

        # Guardar el resultado en una hoja con el nombre del archivo procesado
        sheet_name = file.replace("Resultados_LINTUL2_", "").replace(".xlsx", "")
        merged_df.to_excel(writer, sheet_name=sheet_name[:31], index=False)  # Limitar nombre a 31 caracteres

print(f"Proceso completado. Datos guardados en {output_path}")



Unir en tabla para los modelos

In [6]:
import pandas as pd
import ace_tools as tools

# Cargar datos relevantes
xls = r"C:\Users\Stefania Alvarez\Documents\Universidad\2025-1\Tesis Final\Modelos de prediccion\CordobaEjemplo.xlsx"
clima_df = pd.read_excel(xls, sheet_name="Cordoba1")
toneladas_hora_df = pd.read_excel(xls, sheet_name="1_2000_2020")

# Promedio de precipitación por año desde la hoja horaria
# Ignorar la primera columna "time" si existe
if toneladas_hora_df.columns[0].lower() == 'time':
    precipitacion_hora_df = toneladas_hora_df.iloc[:, 1:]

# Calcular promedios por columna (cada columna representa un año)
promedios_precipitacion = precipitacion_hora_df.mean()

# Extraer solo las columnas necesarias de clima_df
columnas_utiles = {
    "Año": "Ano",
    "Max Temp": "T.Max",
    "Min Temp": "T.Min",
    "Vapor": "P.Vapor",
    "Irradiacion": "Irradicion",
    "Wind Speed": "WindSpeed",
    "Precipitation": "Precipitacion",
    "Toneladas por Hectaria": "Toneladas por Hectaria",  # En caso de que ya esté incluida
    "Departamento": "Departamento"  # En caso de que ya esté incluida
}

# Renombrar si las columnas existen
clima_df = clima_df.rename(columns={col: columnas_utiles[col] for col in columnas_utiles if col in clima_df.columns})

# Insertar las precipitaciones horarias promediadas
clima_df["Precipitacion"] = clima_df["Ano"].map(promedios_precipitacion)

# Agregar columnas faltantes para el formato final
if "Departamento" not in clima_df.columns:
    clima_df["Departamento"] = 1
if "Toneladas por Hectaria" not in clima_df.columns:
    clima_df["Toneladas por Hectaria"] = 0.0  # Placeholder si no está

# Reordenar las columnas para que coincidan con la tabla final
columnas_finales = ["Ano", "T.Max", "T.Min", "P.Vapor", "Precipitacion", "Irradicion", "WindSpeed", "Departamento", "Toneladas por Hectaria"]
tabla_final = clima_df[columnas_finales]

# Guardar a un nuevo Excel
output_path = r"C:\Users\Stefania Alvarez\Documents\Universidad\2025-1\Tesis Final\Modelos de prediccion\Tabla_Resumen_Cordoba.xlsx"
tabla_final.to_excel(output_path, index=False)

tools.display_dataframe_to_user(name="Tabla Final de Resumen", dataframe=tabla_final)


ModuleNotFoundError: No module named 'ace_tools'